In [1]:
#copy files from s3 ( to be done only once)
#!aws s3 cp s3://ncw210data/NewClusterFiles/ /home/ubuntu/data/models_data/ --recursive

In [9]:
!mkdir /home/ubuntu/data/models_data/chartdatafiles

In [13]:
from datetime import datetime
def create_charts_data_for_small_files(filename,p,c):
    df_data = pd.read_csv(filename)
    df_data['pub_year'] = df_data['pub_date'].apply(lambda x: datetime.strptime(str(x),'%Y%m%d').year)
    df_data['appl_year'] = df_data['appl_date'].apply(lambda x: datetime.strptime(str(x),'%Y%m%d').year)
    #round the year to nearest half year
    df_data['years_to_publish'] = df_data['number_of_days'].apply(lambda x: round(x/365 * 2.0) / 2.0)
    
    df_publ_years = df_data.groupby(['pub_year']).size().reset_index(name='count')
    json_publ_years = df_publ_years.to_json(orient='records')
    
    df_appl_years = df_data.groupby(['appl_year']).size().reset_index(name='count')
    json_appl_years = df_appl_years.to_json(orient='records')
    
    df_years = df_data.groupby(['years_to_publish']).size().reset_index(name='count')
    json_years = df_years.to_json(orient='records')
    
    outputfilename = '/home/ubuntu/data/models_data/chartdatafiles/chart_cluster_data_'+ str(p) + '_' +str(c) + '.csv' 
    with open(outputfilename, "w") as chart_file:
        chart_file.write(json_publ_years + '\n' + json_appl_years + '\n' + json_years )
    

In [15]:
#create tfidf models for second level clustered files
for p in range(0,10):
    for c in range(0,10):
        filename = '/home/ubuntu/data/models_data/smalldatafiles/small_cluster_data_'+ str(p) + '_' +str(c) + '.csv' 
        create_charts_data_for_small_files(filename,p,c)

In [14]:
# filename = '/home/ubuntu/data/models_data/smalldatafiles/small_cluster_data_3_2.csv' 
# create_charts_data_for_small_files(filename,3,2)

In [16]:
#Copy files to S3
#!aws s3 cp /home/ubuntu/data/models_data/chartdatafiles/ s3://ncw210data/NewClusterFiles/chartdatafiles/  --recursive --only-show-errors

In [17]:
#!aws s3 cp /home/ubuntu/data/models_data/chartdatafiles/ s3://ncw210data/FullTextDataClusters/  --recursive --only-show-errors